In [1]:
import requests
from datasets import load_dataset
from tqdm import tqdm
import json
import dotenv
import os

In [2]:
dotenv.load_dotenv()
API_KEY = []
API_KEY.append(os.getenv("API_KEY2"))
API_KEY.append(os.getenv("API_KEY3"))
API_KEY.append(os.getenv("API_KEY4"))

In [3]:
dataset_name = "kokujin/prompts_1"
model_name = "Gemini_1.5_Flash"
output_dir = f"./results/{model_name}/"

headers = {"Content-Type": "application/json"}

In [4]:
test = load_dataset(dataset_name, split="test")

opt = model_name
dic = {opt: []}

In [5]:
import logging

# Set up logging
log_filename = f"logs.txt"
logging.basicConfig(filename=log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Example log messages
logging.info("Started processing the dataset.")
logging.info(f"Dataset name: {dataset_name}")
logging.info(f"Model name: {model_name}")
logging.info("Finished processing the dataset.")

In [6]:
import time
err_ = []
start = time.time()
opt_ = 0
for i, prompt in tqdm(enumerate(test)):
    if i%1000 == 0 and i != 0:
        opt_ += 1
        logging.info(f"Switched to API key {opt_}")
    url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={API_KEY[opt_]}'
    if i%10 == 0 and i != 0:
        end = time.time()
        time.sleep(90 - (end - start))
        start = time.time()

    tmp = {
        "Prompt": "",
        "Original": "",
        "Prediction": ""
    }
    resp = prompt["Text"].split('### Output: ')
    prompt = resp[0]
    payload = '{"contents": [{"parts":[{"text":"' + prompt + '"}]}]}'
    #print(payload)

    res = requests.post(url, data=payload, headers=headers)
    logging.info(f"Prompt: {res}")
    resp_ = json.loads(res.text)
    if 'error' in resp_:
        if resp_['error']['code'] == 429:
            opt_ += 1
            if opt_ == 3:
                opt_ = 0
            url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={API_KEY[opt_]}'
            res = requests.post(url, data=pa    yload, headers=headers)
            resp_ = json.loads(res.text)
        else:
            print(resp_)
            err_.append(resp_)
            continue
    logging.info(f"Prompt: {resp_}")
    #print(resp_['candidates'][0]['content']['parts'][0]['text'])
    result = resp_['candidates'][0]['content']['parts'][0]['text']
    tmp["Prompt"] = prompt
    tmp["Original"] = resp[1]
    tmp["Prediction"] = result
    dic[opt].append(tmp)

1299it [3:13:54,  3.45s/it]

{'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


1765it [4:24:12,  6.68s/it]

{'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


1766it [4:24:13,  4.97s/it]

{'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


2399it [5:58:59,  8.98s/it]


In [7]:
results = {
    "Prompt": [],
    "Original": [],
    "Prediction": []
}

In [ ]:
def format(val):
    val = val.replace("'", '"')
    #    val = '"' + val + '"'
    return val

In [ ]:

ommited = []
a = []
for i, t in enumerate(dic[opt]):
    try:
        val_p = t["Prediction"]

        val_p = json.loads("}".join(('{' + "{".join(val_p.split('{')[1:])).split('}')[:-1]) + '}')
        
        #print(val_p, '\n aca termina la review')
        #val_p = format(val_p)
        #val_p = val_p + "\"}" if "\"}" not in val_p else val_p

        val = t["Original"]
        val = format(val)
        val = json.loads(val)
        #print(val, '\n aca termina la review')
        results["Prediction"].append(val_p)
        results["Original"].append(val)
        results["Prompt"].append(t["Prompt"])

    
    except Exception as e:
        ommited.append(i)
        #print(e)
        a.append(t["Prediction"])

print(len(ommited), ommited)

JSONDecodeError: Expecting ',' delimiter: line 1 column 606 (char 605)

In [21]:
print(a)

['```json\n{\n  "Design and Display": {\n    "Display Size": "5.5 inches (13.97 cm)",\n    "Resolution": "1080 x 1920 pixels",\n    "Pixel Density": "401ppi",\n    "Display Type": "P-OLED",\n    "Screen Protection": "Corning Gorilla Glass v3",\n    "Dimensions": "149.1mm x 75.3mm x 7.1mm",\n    "Weight": "152g",\n    "Colour Options": ["Red", "Silver"]\n  },\n  "Battery and Connectivity": {\n    "Battery Capacity": "3000 mAh",\n    "Battery Type": "Li-Polymer",\n    "Fast Charging": "Yes, Quick, v2.0",\n    "Network": ["2G", "3G"],\n    "SIM Card Type": "Micro",\n    "Wi-Fi": "Wi-Fi 802.11, a/ac/b/g/n, Direct, Mobile Hotspot",\n    "Bluetooth": "Bluetooth 4.1",\n    "NFC": "Yes",\n    "GPS": "Yes with A-GPS, Glonass",\n    "FM Radio": "Yes",\n    "3.5mm Headphone Jack": "Yes",\n    "Charging Port": "microUSB 2.0"\n  }\n}\n```\n', '```json\n{\n  "Design and Display": {\n    "Display Size": "6.78 inches (17.22 cm)",\n    "Resolution": "1440 x 3200 pixels",\n    "Pixel Density": "518ppi",

In [10]:
print(len(results['Prediction']))

2239


In [11]:
with open(f'Outputs/{model_name}/{model_name}.json', 'w') as f:
    json.dump(results, f, indent=4)